필수 라이브러리 다운
-
### 주요 라이브러리
#### FinanceDataReader : 한국 주식 가격, 미국주식 가격, 지수, 환율, 암호화폐 가격, 종목 리스팅 등 금융 데이터 수집 라이브러리
#### BeaurifulSoup, Requests : 페이지 크롤링 필수 라이브러리
#### matplotlib : 시각화 라이브러리
#### pandas, numpy : 데이터 정제를 위한 라이브러리
### 추가 설정
#### matplotlib 폰트 설정

In [1]:
import FinanceDataReader as fdr
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import seaborn as sns
fname = 'C:\Windows\Fonts\malgun.ttf'
font_name = fm.FontProperties(fname=fname).get_name()
matplotlib.rc('font', family = font_name)

데이터 크롤링
-
### 미리 수집된 반도체 종목 코드를 code_data에 저장
    with open('./csv_origin/only_code.csv', 'r') as f:
        code_data = f.readlines()
        tmp = []
        for line in code_data:
            tmp.extend(line.split(','))
        code_data = tmp
### code_data에 해당하는 종목의 웹페이지를 크롤링 및 저장
    for code in code_data:
        url = 'http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode='+code[:-1]+'&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
        response = requests.get(url)
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
    
        data = []
        table = soup.find(id = 'highlight_B_Q')
        
        header = table.text.strip()
        header_text = header.split(',')
    
        headers = []
        for item in header_text:
            cleaned_item = re.sub(r'\(E\) : Estimate\n컨센서스 추정치\n|,', '', item)
            headers.extend(re.findall(r'\d{4}/\d{2}', cleaned_item))
        data.append(headers[:-3])
        
        for row in table.find('tbody').find_all('tr'):
            tmp = [cell.text.strip() for cell in row.find_all('td')]
            data.append(tmp[:-3])
    
        columns = [th.text.strip() for th in table.find('tbody').find_all('th')]
        columns.insert(0, 'net quarter')
    
        df = pd.DataFrame(data, columns)
        df=df.transpose()
        
        df.to_csv('./data_csv/'+code[:-1]+'.csv', index=False)
1. 기본 베이스 url에서 code를 추출하여 검색
2. 해당 데이터의 테이블을 id로 검색
3. 콤마(',')를 기준으로 리스트를 분할하여 저장
4. 수집된 데이터 내의 필요 없는 값 정제
5. 수집된 데이터 내용 추출
6. data frame으로 변환
7. 전치변환
8. 코드이름.csv로 저장

In [2]:
with open('./csv_origin/only_code.csv', 'r') as f:
    code_data = f.readlines()
    tmp = []
    for line in code_data:
        tmp.extend(line.split(','))  # 각 라인을 쉼표로 분할하여 tmp 리스트에 추가합니다.
    code_data = tmp

for code in code_data:
    url = 'http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode='+code[:-1]+'&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
    response = requests.get(url)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')

    data = []
    table = soup.find(id = 'highlight_B_Q')
    
    header = table.text.strip()
    header_text = header.split(',')

    headers = []
    for item in header_text:
        cleaned_item = re.sub(r'\(E\) : Estimate\n컨센서스 추정치\n|,', '', item)
        headers.extend(re.findall(r'\d{4}/\d{2}', cleaned_item))
    data.append(headers[:-3])
    
    for row in table.find('tbody').find_all('tr'):
        tmp = [cell.text.strip() for cell in row.find_all('td')]
        data.append(tmp[:-3])

    columns = [th.text.strip() for th in table.find('tbody').find_all('th')]
    columns.insert(0, 'net quarter')

    df = pd.DataFrame(data, columns)
    df=df.transpose()
    
    df.to_csv('./data_csv/'+code[:-1]+'.csv', index=False)